first we need to define our model, and first thing is finding frictions and with zero motor acceleration.

In [1]:
# from Digital_twin import DigitalTwin
# import Digital_twin 
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# theta = 1
# theta_dot = 0

# theta_double_dot = digital_twin_instance.get_theta_double_dot(theta, theta_dot)

In [ ]:
# print(digital_twin_instance.delta_t)

In [2]:
# Model parameters
g = 9.8065     # Acceleration due to gravity (m/s^2)
l = 0.4      # Length of the pendulum (m)
# c_air = 0.05    # Air friction coefficient --------> create a list for grid search
# c_c = 0.05      # Coulomb friction coefficient--------> create a list for grid search
a_m = 0.5     # Motor acceleration force transfer coefficient
mc = 0.0       # Mass of the cart (kg)
mp = 1       # Mass of the pendulum (kg)
I = 0.00       # Moment of inertia of the pendulum (kg·m²)
future_motor_accelerations = []
future_motor_positions = []
future_motor_velocities = []
currentmotor_acceleration = 0.
currentmotor_velocity = 0.
time = 0.
R_pulley = 0.05


In [3]:
def get_theta_double_dot(self, theta, theta_dot):
    """
    Lab 1: Model the angular acceleration (theta_double_dot) 
    as a function of theta, theta_dot and the self.currentmotor_acceleration. 
    You should include the following constants as well: c_air, c_c, a_m, l and g. 
    """
    torque_gravity = -(mp * g * l / (I + mp * l**2)) * np.sin(theta)
    torque_air_friction = -(c_air / (I + mp * l**2)) * theta_dot
    torque_coulomb_friction = -(c_c / (I + mp * l**2)) * theta_dot
    xdoubledot = a_m * R_pulley * currentmotor_acceleration
    torque_motor = - (mp * l / (I + mp * l**2)) * xdoubledot * np.cos(theta)        
    angular_acceleration = torque_gravity + torque_air_friction + torque_coulomb_friction + torque_motor
    return angular_acceleration


In [ ]:
# create a list for finding values of c_air and c_c with grid search
c_air_values = np.linspace(0.01, 0.1, 10)
c_c_values = np.linspace(0.01, 0.1, 10)

# define error function
def error(simulated, real):
    return np.sqrt(np.mean((simulated - real)**2))

# create a list to store the errors
errors = []
# create a list to store the best parameters
best_parameters = []

# define function to do grid on get_theta_double_dot based on input parameters 
def grid_search(c_air_values, c_c_values):
    for c_air in c_air_values:
        for c_c in c_c_values:
            # create a list to store the simulated values
            simulated = []
            # create a list to store the real values
            real = []
            # create a list to store the errors
            error_list = []
            # create a list to store the best parameters
            best_parameters = []
            # loop through the time steps
            for i in range(100):
                # get the simulated values
                simulated.append(get_theta_double_dot(theta, theta_dot))
                # get the real values
                real.append(get_theta_double_dot(theta, theta_dot))
                # get the error
                error_list.append(error(simulated, real))
                # get the best parameters
                best_parameters.append((c_air, c_c))
    return best_parameters, error_list

